In [ ]:
def Poisson_2D(cmin, cmax, Nx, Ny, method='jacobi'):
    """
    :param cmin: lower most grid pt
    :param cmax: upper most grid pt
    :param Nx: points in x
    :param Ny: points in y
    :param method: jacobi from linear solver
    :return: mesh potential and time
    """
    def real2grid(x, y, cmin, cmax, Nx, Ny):
        grid_pt = np.zeros(2)
        if x < cmin[0] or x > cmax[0] or y < cmin[1] or y > cmax[1]:
            raise Exception(f'real2grid point {p} is outside boundary of box')
        else:
            delta = np.zeros(2)
            delta[0] = (cmax[0] - cmin[0])/Nx
            delta[1] = (cmax[1] - cmin[1])/Ny
            grid_pt[0] = int((x - cmin[0]) / delta[0])
            grid_pt[1] = int((y - cmin[1]) / delta[1])
        return grid_pt[0], grid_pt[1]

    def grid2real(i, j, cmin, d):
        return cmin[0] + (i + 0.5) * d[0], cmin[1] + (j + 0.5) * d[1]

    def constant_bc(cmin, cmax, Nx, Ny):
        alpha = encode(0, 0, Ny)
        V0 = 0.0
        V1 = 5e-4
        bc = {alpha:V0}

        for i in range(Nx):
            alpha = encode(i, 0, Ny)
            bc.update({alpha:V0})
            alpha = encode(i, Ny - 1, Ny)
            bc.update({alpha:V0})

        for j in range(Ny):
            alpha = encode(0, j, Ny)
            bc.update({alpha:V0})
            alpha = encode(Nx - 1, j, Ny)
            bc.update({alpha:V0})

        bc = dict(sorted(bc.items()))
        return bc

    def right_hand_side(cmin, cmax, Nx, Ny, val):
        N = Nx * Ny
        b = np.zeros(N)
        # single charge at 0, 0
        # i, j = real2grid(0, 0, cmin, cmax, Nx, Ny)
        # alpha = encode(i, j, Ny)
        # b[alpha] = -val

        # two opposite charges at -0.5,0 and 0.5,0
        i, j = real2grid(-0.5, 0.0, cmin, cmax, Nx, Ny)
        alpha = encode(i, j, Ny)
        b[alpha] = -val

        i, j = real2grid(0.5, 0.0, cmin, cmax, Nx, Ny)
        alpha = encode(i, j, Ny)
        b[alpha] = val

        return b